# import 

In [1]:
import pandas as pd

# folders

In [3]:
FOLDER = '../../data_handson/titanic/'

# load data

In [10]:
df_train = pd.read_csv(FOLDER + 'train.csv')
df_test = pd.read_csv(FOLDER + 'test.csv')

In [11]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [91]:
ticket_list_train = df_train['Ticket'].value_counts().index
ticket_list_test = df_test['Ticket'].value_counts().index

In [93]:
tik = []
for t in ticket_list_test:
    if t in ticket_list_train:
        tik.append(True)
    else:
        tik.append(False)

In [95]:
sum(tik)

115

In [58]:
def gen_feature(df):
    # one-hot
    sex = pd.get_dummies(df['Sex'])
    pclass = pd.get_dummies(df['Pclass'])
    emb = pd.get_dummies(df['Embarked'])
    age = pd.get_dummies(df['Age'] // 10)

    parch = df['Parch']

    # non-null: 1 null: 0
    cabin = (~df['Cabin'].isnull()).astype(int) 

    df_feature = pd.concat(
        [
            sex,
            pclass,
            parch,
            cabin,
            emb,
            age
        ],
        axis=1
    )

    return df_feature

In [59]:
feat_train = gen_feature(df_train)
feat_test = gen_feature(df_test)

In [79]:
feat_test['8.0'] = 0

In [80]:
from fastFM import sgd
model = sgd.FMClassification(n_iter=100, init_stdev=0.1, l2_reg_w=0, l2_reg_V=0, rank=3, step_size=0.1)

In [81]:
import scipy as sp
X_train  = sp.sparse.csr_matrix(feat_train.values)
y_train = df_train['Survived'].values
y_train = 2 * y_train - 1

In [82]:
model.fit(X_train, y_train)

FMClassification(init_stdev=0.1, l2_reg=None, l2_reg_V=0, l2_reg_w=0,
                 n_iter=100, random_state=123, rank=3, step_size=0.1)

In [83]:
X_train = sp.sparse.csr_matrix(feat_train)
y_val = (model.predict(X_train) + 1) / 2
y_val = y_val.astype(int)

In [84]:
sum(y_val == y_train) / len(y_train)

0.2615039281705948

In [85]:
X_train.shape

(891, 19)

In [86]:
X_test = sp.sparse.csr_matrix(feat_test)
y_pred = (model.predict(X_test) + 1) / 2
y_pred = y_pred.astype(int)

In [87]:
y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,